# Feature Extraction: Lines

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt
import matplotlib.colors as clr

from skimage import io
from skimage.draw import line as draw_line
from skimage.feature import canny
from skimage.transform import hough_line, hough_line_peaks
from skimage.transform import probabilistic_hough_line
from skimage.util import img_as_float32 as img_as_float

In [ ]:
def print_imginfo(I):
    print(type(I))
    print(I.shape, I.dtype)
    print('Data range:', np.min(I), 'to', np.max(I))

In [ ]:
use_test = True

if use_test:
    I1 = np.zeros((200, 200))
    I1[draw_line(25, 25, 150, 50)] = 1.0
    I1[draw_line(25, 25, 25, 160)] = 1.0
    I1[draw_line(25, 160, 150, 50)] = 1.0
else:
    I1 = io.imread("../../images/bikes.jpg", as_gray=True)
    I1 = img_as_float(I1[75:275,200:500])
    I1 = canny(I1, sigma=2, low_threshold=0.10, high_threshold=0.20)

fig, ax = plt.subplots()
plt.imshow(I1, cmap='gray')

## Algorithm: Hough Transform

In [ ]:
angles = np.linspace(-np.pi/2, np.pi/2, 360, endpoint=False)

accum, theta, d = hough_line(I1, theta=angles)
peak_data = hough_line_peaks(accum, theta, d, threshold=0.5*accum.max(), num_peaks=10)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,5))

delta_t = 0.5*np.diff(theta).mean()
delta_d = 0.5*np.diff(d).mean()

xmin = np.rad2deg(theta[0] - delta_t)
xmax = np.rad2deg(theta[-1] + delta_t)
ymax = d[-1] + delta_d
ymin = d[0] - delta_d

axis = [ xmin, xmax, ymax, ymin ]
ax.imshow(accum, norm=clr.PowerNorm(gamma=0.25), extent=axis, cmap='gray')

ax.set_aspect(np.abs((xmax-xmin)/(ymax-ymin)))

ax.set_title('Accumulator')
ax.set_xlabel('Angles (degrees)')
ax.set_ylabel('Distance (pixels)')

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5))

ax[0].imshow(I1, cmap='gray')
ax[0].set_title('Full length lines')
ax[0].set_xlim((0,I1.shape[1]))
ax[0].set_ylim((I1.shape[0],1))
ax[0].set_axis_off()

for count, angle, dist in zip(*peak_data):
    print('%3d %5.1f deg %5.1f' % (count, np.rad2deg(angle), dist))
    u = dist*np.cos(angle)
    v = dist*np.sin(angle)
    ax[0].axline((u,v), slope=np.tan(angle + np.pi/2))
    
lines = probabilistic_hough_line(I1, threshold=50, line_length=10, line_gap=3)

ax[1].imshow(np.zeros_like(I1))
ax[1].set_xlim((0, I1.shape[1]))
ax[1].set_ylim((I1.shape[0], 1))
ax[1].set_title('Probabilistic estimate')
ax[1].set_axis_off()

for line in lines:
    p0, p1 = line
    ax[1].plot((p0[0], p1[0]), (p0[1], p1[1]))
    
plt.tight_layout()
plt.show()